## Downsampling png

In [1]:
import openslide
from PIL import Image
import numpy as np
import tifffile as tiff
import os

def downsample_wsi_to_20x_tiff(svs_path,label, tile_size=4096, downsample_factor=2):
    output_dir = "D:/Aamir Gulzar/KSA_project2/Cancer-detection-classifier/data_preprocessing/TIFF"
    
    slide = openslide.OpenSlide(svs_path)
    level_0_dims = slide.level_dimensions[0]

    # Target dimensions (downsampled by 2 for 20x)
    down_width = level_0_dims[0] // downsample_factor
    down_height = level_0_dims[1] // downsample_factor

    # Create an empty numpy array to store the full downsampled image
    downsampled_array = np.zeros((down_height, down_width, 3), dtype=np.uint8)

    print(f"Downsampling from {level_0_dims} to {(down_width, down_height)}")

    for y in range(0, level_0_dims[1], tile_size):
        for x in range(0, level_0_dims[0], tile_size):
            # Read tile from level 0
            read_w = min(tile_size, level_0_dims[0] - x)
            read_h = min(tile_size, level_0_dims[1] - y)
            tile = slide.read_region((x, y), 0, (read_w, read_h)).convert("RGB")

            # Resize to 20x
            tile_resized = tile.resize((read_w // downsample_factor, read_h // downsample_factor), Image.BILINEAR)
            tile_np = np.array(tile_resized)

            # Calculate paste coordinates
            dx = x // downsample_factor
            dy = y // downsample_factor
            dw = tile_np.shape[1]
            dh = tile_np.shape[0]

            # Paste into main array
            downsampled_array[dy:dy+dh, dx:dx+dw, :] = tile_np

    print("✅ Downsampling complete. Saving as TIFF...")

    # Step 1: Generate subfolder name
    folder_name = f"{os.path.basename(svs_path)[:12]}_{label}"
    
    # Step 2: Full path for the subfolder
    subfolder_path = os.path.join(output_dir, folder_name) #D:/Aamir Gulzar/KSA_project2/data_preprocessing/TIFF/TCGA-A6-2686_MSIH
    
    # Step 3: Make sure the folder exists
    os.makedirs(subfolder_path, exist_ok=True)
    
    output_path = os.path.join(subfolder_path, f"{folder_name}.tiff")
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    tiff.imwrite(output_path, downsampled_array)
    print(f"✅ Saved TIFF: {output_path}")

    return downsampled_array, subfolder_path

In [2]:
def save_preview_png(numpy_image,svs_path,label, max_size=(2000, 2000)):
    output_png_path="./Downsampled"
    output_path = os.path.join(output_png_path, f"{os.path.basename(svs_path)[:12]}_{label}.png")
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # Resize image to preview size
    pil_img = Image.fromarray(numpy_image)
    pil_img.thumbnail(max_size, Image.LANCZOS)
    pil_img.save(output_path)
    print(f"✅ Saved preview PNG: {output_path}")

## Testing downsampling

In [ ]:
# # Your WSI path
slide = openslide.OpenSlide(svs_path)
scale_factor = 16.0  # downsample 40x to 20x
level_0_dim = slide.level_dimensions[0]  # (143752, 57366)

# # Output size at 20x
downsampled_size = (level_0_dim[0] // int(scale_factor), level_0_dim[1] // int(scale_factor))

# # # Run downsampling and save preview
wsi_array, subfolder_path = downsample_wsi_to_20x_tiff(svs_path,label="nonsih")
save_preview_png(wsi_array, svs_path,"nonsih",downsampled_size)

In [ ]:
pathtiff= 'D:\Aamir Gulzar\KSA_project2\data_preprocessing\TIFF\TCGA-A6-2686_nonsih'

source_dir = pathtiff #D:/Aamir Gulzar/WSI_Raw_data/0ecb2d7b-0a69-46ce-80d6-1be65cec3cc7
source_dir_quoted = f"{source_dir}" #D:/Aamir Gulzar/WSI_Raw_data/0ecb2d7b-0a69-46ce-80d6-1be65cec3cc7

flat_dir_quoted = f"{flat_dir}" #./FLAT_DIRECTORY

#***************setting directories for downsampled***************#
output_dir= "./FLAT_DIRECTORY/downsampled"
flat_dir = Path(r"D:\Aamir Gulzar\KSA_project2\data_preprocessing\TIFF\TCGA-A6-2686_nonsih")

# svs_path = os.path.join(source_dir_quoted, svs_file) #D:/Aamir Gulzar/WSI_Raw_data/0ecb2d7b-0a69-46ce-80d6-1be65cec3cc7/TCGA-AA-3524-01Z-00-DX1.b1aae264-87be-4514-8f9d-25660b39caa7.svs
# print(f"🚀 Processing: {svs_file} in {subdir}")

#***************Creating .h5 command***************#
command = [
    'python', './CLAM/create_patches_Updated.py',
    '--source', source_dir_quoted,
    '--save_dir', flat_dir,
    '--patch_size', '1024',
    '--step_size','1024',
    '--seg', '--patch'
]

# Execute the command
        
# # Use Popen to stream output line-by-line
with subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True) as proc:
    for line in proc.stdout:
        print(line, end='')  # stream each line live

#***************Renaming .h5 file***************#
new_h5_path= rename_h5_file(svs_file, label, h5_dir="./FLAT_DIRECTORY/patches")